# Zero-shot image classification

## Zero-shot learning with CLIP

In [ ]:
from datasets import load_dataset
import matplotlib.pyplot as plt

dset = "rajuptvs/ecommerce_products_clip"
dataset = load_dataset(dset)
print(dataset["train"][0]["Description"])
plt.imshow(dataset["train"][0]["image"])
plt.show()

In [ ]:
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

categories = ["shirt", "trousers", "shoes", "dress", "hat", "bag", "watch"]

inputs = processor(text=categories, images=dataset["train"][0]["image"], return_tensors="pt", padding=True)
outputs = model(**inputs)

probs = outputs.logits_per_image.softmax(dim=1)
categories[probs.argmax().item()]

## Automated caption quality assessment

In [ ]:
from torchmetrics.functional.multimodal import clip_score

image = dataset["train"][0]["image"]
description = dataset["train"][0]["Description"]

from torchvision.transforms import ToTensor

image = ToTensor()(image)*255
score = clip_score(image, description, "openai/clip-vit-base-patch32")

print(f"CLIP score: {score}")

# Multi-modal sentiment analysis

## Prompting Vision Language Models (VLMs)

In [ ]:
from datasets import load_dataset

dset = "RealTimeData/bbc_news_alltime"
dataset = load_dataset(dset, '2017-01', split="train")
image = dataset[87]["top_image"]
content = dataset[87]["content"]
print(content)

In [ ]:
from transformers import Qwen2VLForConditionalGeneration
from qwen_vl_utils import process_vision_info

vl_model = Qwen2VLForConditionalGeneration.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", device_map="auto", torch_dtype="auto" )

In [ ]:
from transformers import Qwen2VLProcessor

min_pixels = 224 * 224
max_pixels = 448 * 448
vl_model_processor = Qwen2VLProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

In [ ]:
text_query = f"Is the sentiment of the following content good or bad for the Ford share price: {article_text}. Provide reasoning."
chat_template = [{
  "role": "user",
  "content": [
    {"type": "image", "image": article_image},
    {"type": "text", "text": text_query}
  ]
}]

## Multi-modal sentiment classification with Qwen

In [ ]:
text = vl_model_processor.apply_chat_template(chat_template, tokenize=False, add_generation_prompt=True)
image_inputs, _ = process_vision_info(chat_template)
inputs = vl_model_processor(text=[text], images=image_inputs, padding=True, return_tensors="pt")
generated_ids = vl_model.generate(**inputs, max_new_tokens=500)
generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]

In [ ]:
output_text = vl_model_processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False )
print(output_text[0])